## Group Assignment
### Team Number: XX
### Team Member Names: Derek Tan, Jeff Peng, Yuqian Lin
### Team Strategy Chosen: SAFE

### Abstract

Our portfolio optimization strategy involves the use and implementation of the Modern Portfolio Theory (MPT) and analysis of the Efficient Frontier graphs.

The objective of the portfolio optimization strategy is to maximize the portfolio return while maintaining the minimum portfolio risk. 

Modern Portfolio Theory states that since it is assumed that all investors are risk-adverse, when considering the possible portfolio allocation strategies, the investor will prefer the portfolio that maximizes the possible return while maintaining a given amount of risk.

The Efficient Frontier is a graph that illustrates all possible portfolios portfolio allocation distributions. The x-axis represents the volatility of the portfolio, while the y-axis represents the expected return of the portfolio.

The Efficient Frontier shows the optimized portfolios that offer the highest expected return for a given level of risk and the lowest level of risk for a given level of expected return.

An example of the an Efficient Frontier graph is shown below.

[insert image here]

In [93]:
from IPython.display import display, Math, Latex
from datetime import datetime

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt

In [94]:
# Import Financial Data

tickers = pd.read_csv('Tickers.csv', index_col=False)

start_date = '2018-01-01'
end_date = '2021-10-31'

tickers = ['MSFT', 'AAPL', 'TWTR', 'INTC', 'TSM', 'GOOG', 'AMZN', 'FB', 'NVDA', 'TSLA']

data = yf.download(tickers, start=start_date, end=end_date)

closing_prices = data['Adj Close']

closing_prices.head()

[*********************100%***********************]  10 of 10 completed


,AAPL,AMZN,FB,GOOG,INTC,MSFT,NVDA,TSLA,TSM,TWTR
Date,,,,,,,,,,
2018-01-02,41.188164,1189.010010,181.419998,1065.000000,42.406261,81.848297,49.386845,64.106003,36.381702,24.510000
2018-01-03,41.180988,1204.199951,184.669998,1082.479980,40.967079,82.229210,52.637188,63.450001,36.993679,24.450001
2018-01-04,41.372269,1209.589966,184.330002,1086.400024,40.215805,82.952927,52.914650,62.924000,36.798561,23.990000
2018-01-05,41.843311,1229.140015,186.850006,1102.229980,40.496403,83.981392,53.363068,63.316002,37.658871,24.320000
2018-01-08,41.687889,1246.869995,188.279999,1106.939941,40.496403,84.067078,54.998146,67.281998,37.641144,24.590000


To compare the price fluctuations, we will calculate the daily percent change in the price of each stock. This way, it makes the price fluctuations easier to compare, as the price of the stock itself becomes removed from the equation, and we only consider the magnitude of the price movements in comparison to the stock price.

(Yuqian reword this better for me thanks)

In [95]:
# Calculate percent change

percent_change = closing_prices.pct_change().apply(lambda x: np.log(1+x))

percent_change.head()

,AAPL,AMZN,FB,GOOG,INTC,MSFT,NVDA,TSLA,TSM,TWTR
Date,,,,,,,,,,
2018-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,-0.000174,0.012694,0.017756,0.016280,-0.034527,0.004643,0.063739,-0.010286,0.016681,-0.002451
2018-01-04,0.004634,0.004466,-0.001843,0.003615,-0.018509,0.008763,0.005257,-0.008325,-0.005288,-0.018993
2018-01-05,0.011321,0.016033,0.013579,0.014466,0.006953,0.012322,0.008439,0.006210,0.023110,0.013662
2018-01-08,-0.003721,0.014322,0.007624,0.004264,0.000000,0.001020,0.030181,0.060755,-0.000471,0.011041


Now, we want to calculate the variance of the price changes in each stock. Variance is defined as ...

Next, we will calculate the volatility of each stock through its variance. The volatility of each stock is calculated by ...

To optimally diversify the portfolio, we will measure the covariance of the possible stocks in comparison with each other.

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Derek, Yuqian, Jeff